# HitPredict 1st Milestone
## Data Collection, Visualisation and Feature Engineering

Hit predict will predict the popularity of a song based on some of its musical properties. We used the Spotify DB datasert from kaggle, which contains numerous rows of features of over 230.000 tracks. It was assembled using Spotify's API.

As always we started with importing the libraries that we'll be using.

In [ ]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

The database is in .csv format, we used Pandas' read_csv() function to import it to Python. We visualize the data below.

In [ ]:
data = pd.read_csv('SpotifyFeatures.csv')
print(data.head())
print(data.describe())
print(data.keys())

First and foremost let's check for any 0 data points that we might need to replace:

In [ ]:
print(pd.isnull(data).sum())

Fortunately there are none, let's move on with some of the feature engineering that we have done. 

Most values are numerical and need no preprocessing. We do have to convert however some text based rows into numbers which can be fed to the network afterwards.
Such rows are Key, Mode and Time Signature all of which will be replaced with integers 1 through the number of unique types that the given row may contain. 

I was not familiar with most of these terms, below you will find short descriptions that helped me better understand them.
Time signature: (also known as meter signature, metre signature, or measure signature) is a notational convention used in Western musical notation to specify how many beats (pulses) are contained in each measure (bar), and which note value is equivalent to a beat.
Mode: In the theory of Western music, it is a type of musical scale coupled with a set of characteristic melodic behaviors.

In [ ]:
#key first

list_of_keys = data['key'].unique()
for i in range(len(list_of_keys)):
    data.loc[data['key'] == list_of_keys[i], 'key'] = i
data.sample(10)

#mode second

dataframe.loc[dataframe["mode"] == 'Major', "mode"] = 1
dataframe.loc[dataframe["mode"] == 'Minor', "mode"] = 0
dataframe.sample(10)

#time signature last

list_of_time_signatures = data['time_signature'].unique()
for i in range(len(list_of_time_signatures)):
    data.loc[data['time_signature'] == list_of_time_signatures[i], 'time_signature'] = i
data.sample(10)

Since the outcome will be binary, we had to adjust the popularity label either to 1 or 0. This output may change in the project afterwards to a probability. We chose an arbitrary 25-75 split to do this. (Top 25% are considered popular)

In [ ]:
dataframe.loc[dataframe['popularity'] < 75, 'popularity'] = 0 
dataframe.loc[dataframe['popularity'] >= 75, 'popularity'] = 1
dataframe.loc[dataframe['popularity'] == 1]

Finally we select all the features, which will be fed to the network, and split the data into 60% training 20% validation and 20% test subsets.

In [ ]:
features = ["acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "key", "liveness", 
            "mode", "speechiness", "tempo", "time_signature", "valence"]

train, validate, test = np.split(data.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])